# Segmenting and Clustering Neighborhoods in Toronto

## Part 1. Web Scraping with Beautiful Soup

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

Define the dataframe:

In [2]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods.set_index('PostalCode')
neighborhoods

,PostalCode,Borough,Neighborhood


Get the table of interest using BeautifulSoup:

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
table = BeautifulSoup(source, 'lxml').find('table', class_ = "wikitable")

The first row contains heading, but since it is surrounded by 'th' we can simply omit it by finding content inside 'td's.

Iterate through the table line by line:

In [4]:
na = 'Not assigned'

for line in table.tbody.find_all('tr'):
    items = line.find_all('td') 
    if len(items) > 0 : # for heading this list is empty
        if items[1].text != na : # ignore cells with a borough that is Not assigned
            items[0] = items[0].text # get only text from the fields
            items[1] = items[1].text
            if items[2].text.startswith(na) : # case with 'Queen's Park' 
                items[2] = items[1] 
            else : # just get text and remove '\n' at the end
                items[2] = items[2].text.strip('\n') 
            if items[0] in neighborhoods.index : # if already in dataframe 
                previous = neighborhoods.loc[items[0], 'Neighborhood']
                neighborhoods.at[items[0], 'Neighborhood'] =  previous + ', ' + items[2]
            else: # append new row
                neighborhood = pd.DataFrame([items], columns = column_names, index = [items[0]])
                neighborhoods = neighborhoods.append(neighborhood)

In [5]:
neighborhoods.reset_index(drop = True, inplace = True)

In [6]:
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Get the resulting shape:

In [7]:
neighborhoods.shape

(103, 3)

----
## Part 2. Setting coordinates

### Using geocoder 
__<[REQUEST_DENIED] Google - Geocode [empty]>__

In [8]:
import geocoder

def get_lat_lng(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# check whether geocoder is working
g = geocoder.google('M3A, Toronto, Ontario')
g

<[REQUEST_DENIED] Google - Geocode [empty]>

### Using provided csv file

Downloading provided data:

In [10]:
import wget

coords_file = wget.download('http://cocl.us/Geospatial_data')
coords = pd.read_csv(coords_file).rename(columns={"Postal Code": "PostalCode"})
coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging two dataframes on column 'PostalCode':

In [11]:
neighborhoods = pd.merge(neighborhoods, coords, on = 'PostalCode', how = 'inner', validate = 'one_to_one')
neighborhoods.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


Check shape:

In [12]:
neighborhoods.shape

(103, 5)

----
## Part 3. Exploring and clustering the neighborhoods

In [13]:
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Use geopy library to get the latitude and longitude values of Toronto:

In [14]:
location = Nominatim(user_agent = "toronto_explorer").geocode('Toronto, CA')
latitude = location.latitude
longitude = location.longitude

Create a map of Toronto with all neighborhoods on top:

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Since according to the task we can consider only boroughs that contain the word 'Toronto', let's do this: 

In [16]:
toronto = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop = True)
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Check shape:

In [17]:
toronto.shape

(38, 5)

And create a new map with neighborhoods on top:

In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

__Get venues from Foursquare__

Define Foursquare credentials for API access, and parameters for venues search:

In [19]:
CLIENT_ID = 'Y5MEA4MAOXJQCEBDV0YQUL0NU0M5LTXOSWGCQHAZB3Q2NP5Q'
CLIENT_SECRET = 'D24ITBIK3MHDNQBRSZH2UZBEFAGBIE2E1MWWGTRCFIFAKYZ0'
VERSION = '20190915'
RADIUS = 500
LIMIT = 100

Create a function for requesting information about venues from the Foursquare:

In [20]:
def getNearbyVenues(names, latitudes, longitudes):
    venues_list=[]
    n = 1
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('{}: {}'.format(n, name))
        n = n + 1
              
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, RADIUS, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, lat, lng, 
            v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # create a dataframe out of list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Lat', 'Neighborhood Lng', 
                  'Venue', 'Venue Lat', 'Venue Lng', 'Venue Category']
    
    return(nearby_venues)

And run this function for each selected neighborhood:

In [21]:
# expected: print all toronto neighborhoods -- 38
toronto_venues = getNearbyVenues(names = toronto['Neighborhood'],
                                 latitudes = toronto['Latitude'],
                                 longitudes = toronto['Longitude']
                                )

1: Harbourfront, Regent Park
2: Ryerson, Garden District
3: St. James Town
4: The Beaches
5: Berczy Park
6: Central Bay Street
7: Christie
8: Adelaide, King, Richmond
9: Dovercourt Village, Dufferin
10: Harbourfront East, Toronto Islands, Union Station
11: Little Portugal, Trinity
12: The Danforth West, Riverdale
13: Design Exchange, Toronto Dominion Centre
14: Brockton, Exhibition Place, Parkdale Village
15: The Beaches West, India Bazaar
16: Commerce Court, Victoria Hotel
17: Studio District
18: Lawrence Park
19: Roselawn
20: Davisville North
21: Forest Hill North, Forest Hill West
22: High Park, The Junction South
23: North Toronto West
24: The Annex, North Midtown, Yorkville
25: Parkdale, Roncesvalles
26: Davisville
27: Harbord, University of Toronto
28: Runnymede, Swansea
29: Moore Park, Summerhill East
30: Chinatown, Grange Park, Kensington Market
31: Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
32: CN Tower, Bathurst Quay, Island airport, Harbourfront West, 

Check the size of the resulting dataframe:

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1702, 7)


,Neighborhood,Neighborhood Lat,Neighborhood Lng,Venue,Venue Lat,Venue Lng,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


How many unique categories were found?

In [23]:
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories


__Preparation steps for k-means clustering__

Making one hot encoding for further k-clustering:

In [24]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
# and place it as the first column
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Check shape of a new dataframe:

In [25]:
# expected (1702, 239)
toronto_onehot.shape

(1702, 239)

Now group the dataframe by neighborhood and take the mean of each category:

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.03,...,0.01,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.045455,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,...,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


Check the shape of grouped dataframe:

In [27]:
# expected (38, 239)
toronto_grouped.shape

(38, 239)

__Choosing top venues__

Create a dataframe with top 10 venues for each neighborhood:

In [28]:
# sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Hotel,Burger Joint,American Restaurant,Restaurant,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Farmers Market,Café,Beer Bar,Seafood Restaurant,Comfort Food Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Furniture / Home Store,Burrito Place,Convenience Store,Restaurant,Caribbean Restaurant,Stadium,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Fast Food Restaurant,Burrito Place,Auto Workshop,Spa,Garden,Garden Center,Brewery,Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Lounge,Airport Service,Sculpture Garden,Coffee Shop,Plane,Boat or Ferry,Boutique,Bar,Airport Gate


__Run k-means__

Run k-means to cluster the neighborhoods, let's make 6 clusters:

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

Create a dataframe with cluster marks and top 10 venues:

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center,Historic Site
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Diner,Pizza Place,Sporting Goods Shop,Bookstore,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Breakfast Spot
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Farmers Market,Café,Beer Bar,Seafood Restaurant,Comfort Food Restaurant


And visualize clusters:

In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

__Examining clusters__

Looks like Toronto is mostly uniform in its venues. Examining each cluster:

Cluster 1. Coffee and Cafe

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Park,Café,Pub,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center,Historic Site
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Café,Diner,Pizza Place,Sporting Goods Shop,Bookstore,Italian Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bakery,Cosmetics Shop,Cocktail Bar,Clothing Store,Breakfast Spot
3,East Toronto,0,Health Food Store,Trail,Pub,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Cheese Shop,Farmers Market,Café,Beer Bar,Seafood Restaurant,Comfort Food Restaurant
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Indian Restaurant,Gym / Fitness Center,Sushi Restaurant
6,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Nightclub,Restaurant,Italian Restaurant,Baby Store,Diner,Athletics & Sports
7,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Hotel,Burger Joint,American Restaurant,Restaurant,Cosmetics Shop
8,West Toronto,0,Pharmacy,Supermarket,Bakery,Middle Eastern Restaurant,Music Venue,Park,Pizza Place,Coffee Shop,Café,Brewery
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,History Museum,Restaurant,Sporting Goods Shop


Cluster 2. Park + Tennis

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,1,Park,Tennis Court,Wings Joint,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Cluster 3. Garden

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Garden,Home Service,Pool,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint


Cluster 4. Park + Bus

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Park,Bus Line,Swim School,Wings Joint,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


Cluster 5. Park + Playground

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downtown Toronto,4,Park,Playground,Trail,Building,Wings Joint,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
